In [1]:
import os
import zipfile
%load_ext autoreload

%autoreload 2

In [2]:
import tracemalloc
import psutil
import pprint
import pandas as pd
import uuid
import json
import os
import glob
import re
import sys
from bs4 import NavigableString, BeautifulSoup
from collections import defaultdict
import random
import string
import time

from utils.config import config
from utils.logger.logger import loggerCreator

# ePI Modules
from parse.rulebook.rulebook import StyleRulesDictionary

from parse.extractor.parser import parserExtractor
from match.matchDocument.matchDocument import MatchDocument
from documentAnnotation.documentAnnotation import DocumentAnnotation
from htmlDocTypePartitioner.partition import DocTypePartitioner
from extractContentBetweenHeadings.dataBetweenHeadingsExtractor import DataBetweenHeadingsExtractor
from fhirXmlGenerator.fhirXmlGenerator import FhirXmlGenerator
from fhirService.fhirService import FhirService
from utils.logger.matchLogger import MatchLogger
from languageInfo.documentTypeNames.documentTypeNames import DocumentTypeNames
from listBundle.addAndUpdateListBundle.addAndUpdateListBundle import ListBundleHandler

class FolderNotFoundError(Exception):
    pass

class Metrics:
    
    def __init__(self, logFileName, logger):
        self.logFileName = logFileName
        self.start()
        self.writer = open(self.logFileName, 'a')
        self.writer.write("StepName,Time,Current Memory,Peak Memory,Used Ram Percentage\n")
        self.finalPeak = 0
        self.finalTotalTime = 0
        self.finalUsedRamPerc = 0
        self.logger = logger
    
    def start(self):
        self.startTime = time.time()
        tracemalloc.start()
    
    def getMetric(self, msg):
        
        self.endTime = time.time()
        
        self.totalTime = self.endTime - self.startTime
        
        
        current, peak = tracemalloc.get_traced_memory()
        current = current / 10**6
        peak = peak / 10**6
        
        usedRamPerc = psutil.virtual_memory()[2]
        
        self.finalPeak = max(self.finalPeak, peak)
        self.finalUsedRamPerc = max(self.finalUsedRamPerc, usedRamPerc)

        self.finalTotalTime = self.finalTotalTime + self.totalTime
        #self.finalTotalTime = round(self.finalTotalTime/60,3)
        
        outputString = f"{msg},{round(self.totalTime/60,4)} Min,{current} MB,{peak} MB,{usedRamPerc}%\n"
        
        self.logger.logFlowCheckpoint(f"{outputString}")
        
        print(f"Metrics : {outputString}")
        self.writer.write(outputString)
        tracemalloc.stop()
        tracemalloc.start()
        self.startTime = time.time()
    def end(self):
        
        current, peak = tracemalloc.get_traced_memory()
        current = current / 10**6
        outputString = f"Final Metrics,{round(self.finalTotalTime/60,4)} Min,{current} MB,{self.finalPeak} MB,{self.finalUsedRamPerc}%\n"
        print(f"Metrics : {outputString}")
        self.logger.logFlowCheckpoint(f"{outputString}")
        self.writer.write(outputString)
        self.writer.close()
        tracemalloc.stop()
        
        


def convertToInt(x):
    try:
        return str(int(x))
    except:
        return x


def convertCollectionToDataFrame(collection):

    dfExtractedHier = pd.DataFrame(collection)
    dfExtractedHier['parent_id'] = dfExtractedHier['parent_id'].apply(
        lambda x: convertToInt(x))
    dfExtractedHier['id'] = dfExtractedHier['id'].apply(
        lambda x: convertToInt(x))

    return dfExtractedHier

def getRandomString(N):
    str_ = ''.join(random.choice(string.ascii_uppercase + string.digits
                                 + string.ascii_lowercase) for _ in range(N))
    return str_


def convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog):

    module_path = os.path.join(basePath)

    if "/" in basePath:
        pathSep = "/"
    else:
        pathSep = "\\"
    
    # Generate output folder path
    output_json_path = os.path.join(basePath, 'outputJSON')

    """
        Check if input folder exists, else throw exception
    """
    if(os.path.exists(module_path)):
        filenames = glob.glob(os.path.join(module_path, htmlDocName))

        # Create language specific folder in outputJSON folder if it doesn't exist
        if(not os.path.exists(output_json_path)):
            os.mkdir(output_json_path)
        logger = MatchLogger(f'Parser_{getRandomString(1)}', htmlDocName,
                             domain, procedureType, languageCode, "HTML", fileNameLog)

        styleLogger = MatchLogger(
            f'Style Dictionary_{getRandomString(1)}', htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)

        styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                             controlBasePath=controlBasePath,
                                             language=languageCode,
                                             fileName=fileNameQrd,
                                             domain=domain,
                                             procedureType=procedureType
                                             )

        parserObj = parserExtractor(config, logger, styleRulesObj.styleRuleDict,
                                    styleRulesObj.styleFeatureKeyList,
                                    styleRulesObj.qrd_section_headings)
        
        for input_filename in filenames:
          # if(input_filename.find('Kalydeco II-86-PI-clean')!=-1):
            output_filename = os.path.join(output_json_path, htmlDocName)
            style_filepath =  output_filename.replace('.html','.txt')
            style_filepath =  style_filepath.replace('.txtl','.txt')
            style_filepath =  style_filepath.replace('.htm','.txt')
            print("-------------",style_filepath,"-----------------")

            output_filename = output_filename.replace('.html', '.json')
            output_filename = output_filename.replace('.htm', '.json')
            print(input_filename, output_filename)
            parserObj.createPIJsonFromHTML(input_filepath=input_filename,
                                           output_filepath=output_filename,
                                           style_filepath = style_filepath,
                                           img_base64_dict=parserObj.convertImgToBase64(input_filename)
                                           )
        #return parserObj, input_filename, output_filename, style_filepath
        return output_filename.split(pathSep)[-1], style_filepath
    else:
        try:    
            raise FolderNotFoundError(module_path + " not found")
        except:  
            logger.logFlowCheckpoint("Folder For Language Code Not Found In Input File")
            logger.logException("Folder For Language Code Not Found In Input File")
        raise FolderNotFoundError(module_path + " not found")
        return None


def splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog):

    styleLogger = MatchLogger(
        f'Style Dictionary_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    styleRulesObj = StyleRulesDictionary(logger=styleLogger,
                                        controlBasePath=controlBasePath,
                                        language=languageCode,
                                        fileName=fileNameQrd,
                                        domain=domain,
                                        procedureType=procedureType
                                        )
    
    path_json = os.path.join(basePath,'outputJSON', fileNameJson)
    print("PathJson",path_json)
    partitionLogger = MatchLogger(
        f'Partition_{getRandomString(1)}', fileNameJson, domain, procedureType, languageCode, "Json", fileNameLog)

    partitioner = DocTypePartitioner(partitionLogger, domain, procedureType)

    partitionedJsonPaths = partitioner.partitionHtmls(
        styleRulesObj.qrd_section_headings, path_json)

    return partitionedJsonPaths


def extractAndValidateHeadings(controlBasePath,
                                basePath,
                                domain,
                                procedureType,
                                languageCode,
                                documentNumber,
                                fileNameDoc,
                                fileNameQrd,
                                fileNameMatchRuleBook,
                                fileNameDocumentTypeNames,
                                fileNameLog,
                                stopWordFilterLen=6,
                                isPackageLeaflet=False,
                                medName=None
                                ):

    if documentNumber == 0:
        topHeadingsConsidered = 4
        bottomHeadingsConsidered = 6
    elif documentNumber == 1:
        topHeadingsConsidered = 3
        bottomHeadingsConsidered = 5
    elif documentNumber == 2:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 15
    else:
        topHeadingsConsidered = 5
        bottomHeadingsConsidered = 10

    print(f"Starting Heading Extraction For File :- {fileNameDoc}")
    logger = MatchLogger(f"Heading Extraction {fileNameDoc}_{getRandomString(1)}", fileNameDoc, domain, procedureType, languageCode, documentNumber, fileNameLog)
    logger.logFlowCheckpoint("Starting Heading Extraction")

    stopWordlanguage = DocumentTypeNames(
        controlBasePath=controlBasePath,
        fileNameDocumentTypeNames=fileNameDocumentTypeNames,
        languageCode=languageCode,
        domain=domain,
        procedureType=procedureType,
        documentNumber=documentNumber
        ).extractStopWordLanguage()

    matchDocObj = MatchDocument(
        logger,
        controlBasePath,
        basePath,
        domain,
        procedureType,
        languageCode,
        documentNumber,
        fileNameDoc,
        fileNameQrd,
        fileNameMatchRuleBook,
        fileNameDocumentTypeNames,
        topHeadingsConsidered,
        bottomHeadingsConsidered,
        stopWordFilterLen,
        stopWordlanguage,
        isPackageLeaflet,
        medName)
    df, coll, documentType, documentTypeForUI = matchDocObj.matchHtmlHeaddingsWithQrd()

    return df, coll, documentType, documentTypeForUI


def parseDocument(controlBasePath,
                  basePath,
                  htmlDocName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName = None):
    
    listRegulatedAuthCodesAccrossePI = []
    
    if "/" in basePath:
        pathSep = "/"        
    else:
        pathSep = "\\"
    
    fileNameLog = os.path.join(basePath,'FinalLog.txt')

    pathComponents = basePath.split(pathSep)
    print(pathComponents, htmlDocName)
    timestamp = pathComponents[-1]
    languageCode =  pathComponents[-2]
    medName = pathComponents[-3]
    procedureType = pathComponents[-4]
    domain = pathComponents[-5]

    print(timestamp, languageCode, medName, procedureType, domain)
        
    flowLogger =  MatchLogger(f"Flow Logger HTML_{getRandomString(1)}", htmlDocName, domain, procedureType, languageCode, "HTML", fileNameLog)
    
    metrics = Metrics(os.path.join(basePath,'Metrics.csv'),flowLogger)
    
    
    flowLogger.logFlowCheckpoint("Starting HTML Conversion To Json")
    ###Convert Html to Json
    
    fileNameJson, stylesFilePath = convertHtmlToJson(controlBasePath, basePath, domain, procedureType, languageCode, htmlDocName, fileNameQrd, fileNameLog)
    
    print("stylePath:-",stylesFilePath)
    flowLogger.logFlowCheckpoint("Completed HTML Conversion To Json")
    metrics.getMetric("HTML Conversion To Json")

    flowLogger.logFlowCheckpoint("Starting Json Split")

    ###Split Uber Json to multiple Jsons for each category.
    partitionedJsonPaths = splitJson(controlBasePath, basePath, domain, procedureType, languageCode, fileNameJson, fileNameQrd, fileNameLog)
    
    partitionedJsonPaths = [ path.split(pathSep)[-1] for path in partitionedJsonPaths]
    flowLogger.logFlowCheckpoint(str(partitionedJsonPaths))
    
    flowLogger.logFlowCheckpoint("Completed Json Split")
    metrics.getMetric("Split Json")
    
    flowLogger.logFlowCheckpoint("Started Processing Partitioned Jsons")
    
    for index, fileNamePartitioned in enumerate(partitionedJsonPaths):
        print("Index", index)
        #if index in [1,2,3]:
        #    continue
        
        flowLogger.logFlowCheckpoint(f"\n\n\n\n||||||||||||||||||||||||||||||||{str(index)} ||||| {str(fileNamePartitioned)}||||||||||||||||||||||||||||||||\n\n\n\n")
        
        if index == 3:
            stopWordFilterLen = 100
            isPackageLeaflet = True
        else:
            stopWordFilterLen = 6
            isPackageLeaflet = False
            
        df, coll, documentType, documentTypeForUI = extractAndValidateHeadings(controlBasePath,
                                    basePath,
                                    domain,
                                    procedureType,
                                    languageCode,
                                    index,
                                    fileNamePartitioned,
                                    fileNameQrd,
                                    fileNameMatchRuleBook,
                                    fileNameDocumentTypeNames,
                                    fileNameLog,
                                    stopWordFilterLen=stopWordFilterLen,
                                    isPackageLeaflet=isPackageLeaflet,
                                    medName=medName)
        #return df, coll, documentType, documentTypeForUI
        print(f"Completed Heading Extraction For File")
        flowLogger.logFlowCheckpoint("Completed Heading Extraction For File")
        metrics.getMetric(f"{index}: Heading Extraction")

        print(f"Starting Document Annotation For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Document Annotation For File")
        documentAnnotationObj = DocumentAnnotation(fileNamePartitioned,'c20835db4b1b4e108828a8537ff41506','f54d15d1e39543d5a4843f4a6ed6c328',"https://spor-sit.azure-api.net","/pms/api/v2/","/sms/api/v2/",df,coll, index)
        #try
        #    pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
        #    print(pms_oms_annotation_data)
        #except Exception as e:
        #    pms_oms_annotation_data = None
        #    print("Error Found", str(e))
        pms_oms_annotation_data = documentAnnotationObj.processRegulatedAuthorizationForDoc()
        print(pms_oms_annotation_data)
        
        
        print(f"Completed Document Annotation")        
        flowLogger.logFlowCheckpoint("Completed Document Annotation")
        metrics.getMetric(f"{index}: Document Annotation")
        
        print(f"Starting Extracting Content Between Heading For File :- {fileNamePartitioned}")        
        flowLogger.logFlowCheckpoint("Starting Extracting Content Between Heading")
        
        extractContentlogger =  MatchLogger(f'ExtractContentBetween_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        extractorObj = DataBetweenHeadingsExtractor(extractContentlogger, basePath, coll)
        dfExtractedHierRR = extractorObj.extractContentBetweenHeadings(fileNamePartitioned)
        
        print(f"Completed Extracting Content Between Heading")        
        flowLogger.logFlowCheckpoint("Completed Extracting Content Between Heading")
        metrics.getMetric(f"{index}: Content Extraction")
        
        
        xmlLogger =  MatchLogger(f'XmlGeneration_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        
        listBundleDocumentTypeCodesFilePath = os.path.join(controlBasePath,
                                                                listBundleDocumentTypeCodesFileName.split(".")[0],
                                                                listBundleDocumentTypeCodesFileName)
        with open(listBundleDocumentTypeCodesFilePath, encoding='utf-8') as f:
            listBundleDocumentTypeCodes = json.load(f)
            
        bundleDocumentTypeCode = listBundleDocumentTypeCodes[domain][str(index)]['listBundleCode']
        bundleMetaData = {'pmsOmsAnnotationData':pms_oms_annotation_data,
                          'documentTypeCode': bundleDocumentTypeCode,
                          'documentType': documentTypeForUI,
                          'medName': medName}
        
        fhirXmlGeneratorObj = FhirXmlGenerator(xmlLogger, controlBasePath, basePath, bundleMetaData, stylesFilePath)
        fileNameXml = fileNamePartitioned.replace('.json','.xml')
        generatedXml = fhirXmlGeneratorObj.generateXml(dfExtractedHierRR, fileNameXml)
        
        metrics.getMetric(f"{index}: Generate XML")
        
        fhirServiceLogger =  MatchLogger(f'XML Submission Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        
        fhirServiceObj = FhirService(fhirServiceLogger, apiMmgtBaseUrl, addBundleApiEndPointUrlSuffix, apiMmgtSubsKey, basePath, generatedXml)
        fhirServiceObj.submitFhirXml()
        
        
        
        
        metrics.getMetric(f"{index}: Submit FHIR Msg")
        
        print(f"Created XML File For :- {fileNamePartitioned}")
        
        flowLogger.logFlowCheckpoint("Starting list bundle update/addition")
        if documentAnnotationObj.listRegulatedAuthorizationIdentifiers != None:
            for id in documentAnnotationObj.listRegulatedAuthorizationIdentifiers:
                listRegulatedAuthCodesAccrossePI.append(id)
        listBundleLogger =  MatchLogger(f'List Bundle Creation Logger_{index}_{getRandomString(1)}', fileNamePartitioned, domain, procedureType, languageCode, index, fileNameLog)
        print("\nlistRegulatedAuthCodesAccrossePI",listRegulatedAuthCodesAccrossePI)
        #try:
        listBundleHandler = ListBundleHandler(listBundleLogger,
                 domain,
                 procedureType,
                 index,
                 documentType,
                 documentTypeForUI,
                 languageCode,
                 medName,
                 controlBasePath,
                 jsonTempFileName,
                 listBundleDocumentTypeCodesFileName,
                 fileNameDocumentTypeNames,
                 listRegulatedAuthCodesAccrossePI,
                 apiMmgtBaseUrl,
                 getListApiEndPointUrlSuffix,
                 addUpdateListApiEndPointUrlSuffix,
                 apiMmgtSubsKey)

        listBundleXml = listBundleHandler.addOrUpdateDocumentItem(str(fhirServiceObj.SubmittedFhirMsgRefId), pms_oms_annotation_data)
        print(listBundleXml)
        listBundleHandler.submitListXmLToServer(listBundleXml)

        flowLogger.logFlowCheckpoint("Completed list bundle update/addition")
        metrics.getMetric(f"{index}: Update/Add List Bundle")
            #return df,coll,dfExtractedHierRR
        #except Exception as e:
        #    print(str(e))
        #    if 'No MAN Code found' in str(e):
        #        flowLogger.logFlowCheckpoint("Skipping list bundle addtion/update as no MAN found")
            
    
    flowLogger.logFlowCheckpoint("Completed Processing Partitioned Jsons")
    metrics.getMetric(f"{index}: Completed")
    metrics.end()

C:\Users\vipsharm\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vipsharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#from wordToHtmlConvertor.wordToHtmlConvertor import WordToHtmlConvertor
#
#wordToHtmlConvertorObj = WordToHtmlConvertor()
#wordToHtmlConvertorObj.convertWordToHTML()

In [4]:
def runAll(inputList):
    for inputDoc in inputList:
        # inputZipFolderPath = "F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\\inputblob"
        inputZipFolderPath = os.path.abspath(os.path.join('..'))
        inputZipFolderPath = os.path.join(inputZipFolderPath, 'inputblob')
        inputZipFileName = inputDoc

        fileNameQrd = 'qrd_canonical_model.csv'
        fileNameMatchRuleBook = 'ruleDict.json'
        fileNameDocumentTypeNames = 'documentTypeNames.json'
        fsMountName = '/mounted'
        jsonTempFileName = 'listBundleJsonTemplate.json'
        listBundleDocumentTypeCodesFileName = 'listBundleDocumentTypeCodes.json'
        apiMmgtBaseUrl = "https://ema-dap-epi-dev-fhir-apim.azure-api.net"
        getListApiEndPointUrlSuffix = "/epi/v1/List"
        addUpdateListApiEndPointUrlSuffix = "/epi-w/v1/List"
        addBundleApiEndPointUrlSuffix = "/epi-w/v1/Bundle"
        apiMmgtSubsKey = "ba6d7e9a73ed4facaa58fc983bf6db50"



        info = inputZipFileName.split("~")

        try:
            medName = info[0]
            domain = info[1]
            procedureType = info[2]
            languageCode = info[3]
            timestamp = info[4]
            timestamp = timestamp.replace(".zip","")

        except Exception:
            raise f"Missing required info in the zip file name {inputZipFileName}"

        if "\\" in os.getcwd():
            localEnv = True
            inputZipFolderPath = os.path.join(os.path.abspath(os.path.join('..')),inputZipFolderPath)
            outputFolderPath = os.path.join(os.path.abspath(os.path.join('..')), 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(os.path.abspath(os.path.join('..')),'control')
        else:
            localEnv = False
            inputZipFolderPath = os.path.join(f'{fsMountName}',inputZipFolderPath)
            outputFolderPath = os.path.join(f'{fsMountName}', 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(f'{fsMountName}','control')


        print(inputZipFileName, inputZipFolderPath, outputFolderPath, controlFolderPath)

        mode = 0o666

        if localEnv is True:
            inputZipFolderPath = inputZipFolderPath.replace("/","\\")
            outputFolderPath = outputFolderPath.replace("/","\\")
            controlFolderPath = controlFolderPath.replace("/","\\")

        try:
            os.makedirs(inputZipFolderPath, mode)
            os.makedirs(outputFolderPath, mode)
            os.makedirs(controlFolderPath, mode)

        except Exception:
            print("Already Present")

        with zipfile.ZipFile(f'{inputZipFolderPath}/{inputZipFileName}',"r") as zip_ref:
                zip_ref.extractall(outputFolderPath)


        _,_,fileNames = next(os.walk(outputFolderPath))
        htmlFileName = [fileName for fileName in fileNames if ".htm" in fileName][0]

        print(htmlFileName)

        parseDocument(controlFolderPath,
                  outputFolderPath,
                  htmlFileName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName)


In [5]:
def runAllTest(inputList):
    for inputDoc in inputList:
        # inputZipFolderPath = "F:\Projects\EMA\Repository\EMA EPI PoC\\function_code\\inputblob"
        inputZipFolderPath = os.path.abspath(os.path.join('..'))
        inputZipFolderPath = os.path.join(inputZipFolderPath, 'inputblob')
        inputZipFileName = inputDoc
        
        fileNameQrd = 'qrd_canonical_model.csv'
        fileNameMatchRuleBook = 'ruleDict.json'
        fileNameDocumentTypeNames = 'documentTypeNames.json'
        fsMountName = '/mounted'
        jsonTempFileName = 'listBundleJsonTemplate.json'
        listBundleDocumentTypeCodesFileName = 'listBundleDocumentTypeCodes.json'
        apiMmgtBaseUrl = "https://ema-dap-epi-tst-fhir-apim.azure-api.net"
        getListApiEndPointUrlSuffix = "/epi/v1/List"
        addUpdateListApiEndPointUrlSuffix = "/epi-w/v1/List"
        addBundleApiEndPointUrlSuffix = "/epi-w/v1/Bundle"
        apiMmgtSubsKey = "9e9d47b8a08148f9833e17462d90574a"




        info = inputZipFileName.split("~")

        try:
            medName = info[0]
            domain = info[1]
            procedureType = info[2]
            languageCode = info[3]
            timestamp = info[4]
            timestamp = timestamp.replace(".zip","")

        except Exception:
            raise f"Missing required info in the zip file name {inputZipFileName}"

        if "\\" in os.getcwd():
            localEnv = True
            inputZipFolderPath = os.path.join(os.path.abspath(os.path.join('..')),inputZipFolderPath)
            outputFolderPath = os.path.join(os.path.abspath(os.path.join('..')), 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(os.path.abspath(os.path.join('..')),'control')
        else:
            localEnv = False
            inputZipFolderPath = os.path.join(f'{fsMountName}',inputZipFolderPath)
            outputFolderPath = os.path.join(f'{fsMountName}', 'work', f"{domain}", f"{procedureType}", f"{medName}", f"{languageCode}", f"{timestamp}")
            controlFolderPath = os.path.join(f'{fsMountName}','control')


        print(inputZipFileName, inputZipFolderPath, outputFolderPath, controlFolderPath)

        mode = 0o666

        if localEnv is True:
            inputZipFolderPath = inputZipFolderPath.replace("/","\\")
            outputFolderPath = outputFolderPath.replace("/","\\")
            controlFolderPath = controlFolderPath.replace("/","\\")

        try:
            os.makedirs(inputZipFolderPath, mode)
            os.makedirs(outputFolderPath, mode)
            os.makedirs(controlFolderPath, mode)

        except Exception:
            print("Already Present")

        with zipfile.ZipFile(f'{inputZipFolderPath}/{inputZipFileName}',"r") as zip_ref:
                zip_ref.extractall(outputFolderPath)


        _,_,fileNames = next(os.walk(outputFolderPath))
        htmlFileName = [fileName for fileName in fileNames if ".htm" in fileName][0]

        print(htmlFileName)

        parseDocument(controlFolderPath,
                  outputFolderPath,
                  htmlFileName,
                  fileNameQrd,
                  fileNameMatchRuleBook,
                  fileNameDocumentTypeNames,
                  jsonTempFileName,
                  listBundleDocumentTypeCodesFileName,
                  apiMmgtBaseUrl,
                  getListApiEndPointUrlSuffix,
                  addUpdateListApiEndPointUrlSuffix,
                  addBundleApiEndPointUrlSuffix,
                  apiMmgtSubsKey,
                  medName)


In [44]:

inputList = ['ELOCTA~H~CAP~de~2021-06-09T09-36-33Z.zip']

runAll(inputList)

2021-06-24 02:27:08,276 : Flow Logger HTML_y : Starting HTML Conversion To Json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:08,276 : Flow Logger HTML_y : Starting HTML Conversion To Json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:08,286 : Style Dictionary_C : Reading style dictionary in file: rule_dictionary_de.json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:08,321 : Style Dictionary_C : Qrd Section Keys Retrieved For Style Dictionary: ZUSAMMENFASSUNG DER MERKMALE DES ARZNEIMITTELS, ANHANG II, A. ETIKETTIERUNG , B. PACKUNGSBEILAGE | H | CAP |  de | HTML | Elocta_clean.htm


Elocta~H~CAP~de~2021-06-09T09-36-33Z.zip D:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z D:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
Elocta_clean.htm
['D:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'Elocta', 'de', '2021-06-09T09-36-33Z'] Elocta_clean.htm
2021-06-09T09-36-33Z de Elocta CAP H
------------- D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.txt -----------------
D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\Elocta_clean.htm D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.json


2021-06-24 02:27:09,298 : Parser_A : Style Information Stored In File: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.txt | H | CAP |  de | HTML | Elocta_clean.htm


!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!


2021-06-24 02:27:15,240 : Parser_A : Writing to file: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\Elocta\de\2021-06-09T09-36-33Z\outputJSON\Elocta_clean.json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:15,751 : Flow Logger HTML_y : Completed HTML Conversion To Json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:15,751 : Flow Logger HTML_y : Completed HTML Conversion To Json | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:15,756 : Flow Logger HTML_y : HTML Conversion To Json,0.1247 Min,7.307077 MB,19.269271 MB,68.1%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:15,756 : Flow Logger HTML_y : HTML Conversion To Json,0.1247 Min,7.307077 MB,19.269271 MB,68.1%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:15,770 : Flow Logger HTML_y : Starting Json Split | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:27:15,770 : Flow Logger HTML_y : Starting Json Split | H | CAP |  de | HTML | Elocta_clean.htm
2021

...

POST sucessful: XML added with id 3bf85b2b-b227-4ffc-ac05-3ec7f67e6179
Metrics : 3: Submit FHIR Msg,0.0755 Min,0.062626 MB,2.895397 MB,71.5%

Created XML File For :- Elocta_clean_ PACKUNGSBEILAGE.json

listRegulatedAuthCodesAccrossePI ['EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008', 'EU/1/15/1046/001', 'EU/1/15/1046/002', 'EU/1/15/1046/003', 'EU/1/15/1046/004', 'EU/1/15/1046/005', 'EU/1/15/1046/006', 'EU/1/15/1046/007', 'EU/1/15/1046/008']


2021-06-24 02:32:17,993 : List Bundle Creation Logger_3_h : Getting list bundle for MAN EU/1/15/1046/002  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:19,181 : List Bundle Creation Logger_3_h : Getting list bundle for MAN EU/1/15/1046/003  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:20,378 : List Bundle Creation Logger_3_h : Getting list bundle for MAN EU/1/15/1046/004  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:21,346 : List Bundle Creation Logger_3_h : Getting list bundle for MAN EU/1/15/1046/005  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:22,202 : List Bundle Creation Logger_3_h : Getting list bundle for MAN EU/1/15/1046/006  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:23,054 : List Bundle Creation Logger_3_h : Getting list bundle for MAN EU/1/15/1046/007  | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:24,03

b'<List xmlns="http://hl7.org/fhir"><id value="73beb5e0-2d74-4b79-93f4-b2c5450bfe38" /><identifier><system value="http://spor.ema.europa.eu/v2/medicine-name" /><value value="elocta" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/001" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/002" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/003" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/004" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/1046/005" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/15/

2021-06-24 02:32:34,171 : List Bundle Creation Logger_3_h : List update successfully completed 73beb5e0-2d74-4b79-93f4-b2c5450bfe38 | H | CAP |  de | 3 | Elocta_clean_ PACKUNGSBEILAGE.json
2021-06-24 02:32:34,174 : Flow Logger HTML_y : Completed list bundle update/addition | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:32:34,174 : Flow Logger HTML_y : Completed list bundle update/addition | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:32:34,178 : Flow Logger HTML_y : 3: Update/Add List Bundle,0.2869 Min,0.300442 MB,0.455926 MB,72.2%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:32:34,178 : Flow Logger HTML_y : 3: Update/Add List Bundle,0.2869 Min,0.300442 MB,0.455926 MB,72.2%
 | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:32:34,184 : Flow Logger HTML_y : Completed Processing Partitioned Jsons | H | CAP |  de | HTML | Elocta_clean.htm
2021-06-24 02:32:34,184 : Flow Logger HTML_y : Completed Processing Partitioned Jsons | H | CAP |  de | HTML | Eloc

Metrics : 3: Update/Add List Bundle,0.2869 Min,0.300442 MB,0.455926 MB,72.2%

Metrics : 3: Completed,0.0001 Min,0.003357 MB,0.152221 MB,72.2%

Metrics : Final Metrics,5.4293 Min,0.0 MB,20.035667 MB,72.2%



In [51]:


inputList = ['GONAL-f~H~CAP~en~2021-06-04T08-55-59Z.zip']

runAll(inputList)

2021-06-24 03:03:22,995 : Flow Logger HTML_1 : Starting HTML Conversion To Json | H | CAP |  en | HTML | gonal-f_clean.htm
2021-06-24 03:03:23,004 : Style Dictionary_p : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | gonal-f_clean.htm


GONAL-f~H~CAP~en~2021-06-04T08-55-59Z.zip D:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\en\2021-06-04T08-55-59Z D:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
gonal-f_clean.htm
['D:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'GONAL-f', 'en', '2021-06-04T08-55-59Z'] gonal-f_clean.htm
2021-06-04T08-55-59Z en GONAL-f CAP H


2021-06-24 03:03:23,046 : Style Dictionary_p : Qrd Section Keys Retrieved For Style Dictionary: SUMMARY OF PRODUCT CHARACTERISTICS, ANNEX II, A. LABELLING , B. PACKAGE LEAFLET | H | CAP |  en | HTML | gonal-f_clean.htm


------------- D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\en\2021-06-04T08-55-59Z\outputJSON\gonal-f_clean.txt -----------------
D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\en\2021-06-04T08-55-59Z\gonal-f_clean.htm D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\en\2021-06-04T08-55-59Z\outputJSON\gonal-f_clean.json


2021-06-24 03:03:24,882 : Parser_t : Style Information Stored In File: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\GONAL-f\en\2021-06-04T08-55-59Z\outputJSON\gonal-f_clean.txt | H | CAP |  en | HTML | gonal-f_clean.htm


...

HTTP error occurred: 413 Client Error: Payload Too Large for url: https://ema-dap-epi-dev-fhir-apim.azure-api.net/epi-w/v1/Bundle
Error log: The requested resource exceeded the backing database's size limit.
Metrics : 3: Submit FHIR Msg,0.2155 Min,0.305784 MB,0.318887 MB,76.6%

Created XML File For :- gonal-f_clean_ PACKAGE LEAFLET.json

listRegulatedAuthCodesAccrossePI ['EU/1/95/001/025', 'EU/1/95/001/026', 'EU/1/95/001/027', 'EU/1/95/001/021', 'EU/1/95/001/031', 'EU/1/95/001/000', 'EU/1/95/001/033', 'EU/1/95/001/034', 'EU/1/95/001/035', 'EU/1/95/001/025', 'EU/1/95/001/026', 'EU/1/95/001/027', 'EU/1/95/001/021', 'EU/1/95/001/031', 'EU/1/95/001/000', 'EU/1/95/001/033', 'EU/1/95/001/034', 'EU/1/95/001/035']


2021-06-24 03:14:57,040 : List Bundle Creation Logger_3_T : Getting list bundle for MAN EU/1/95/001/026  | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:14:57,924 : List Bundle Creation Logger_3_T : Getting list bundle for MAN EU/1/95/001/027  | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:14:58,771 : List Bundle Creation Logger_3_T : Getting list bundle for MAN EU/1/95/001/021  | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:14:59,624 : List Bundle Creation Logger_3_T : Getting list bundle for MAN EU/1/95/001/031  | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:15:00,493 : List Bundle Creation Logger_3_T : Getting list bundle for MAN EU/1/95/001/000  | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:15:01,375 : List Bundle Creation Logger_3_T : Getting list bundle for MAN EU/1/95/001/033  | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:15:02,21

b'<List xmlns="http://hl7.org/fhir"><id value="9b70aea6-c211-43a1-8204-2c36aaea490d" /><identifier><system value="http://spor.ema.europa.eu/v2/medicine-name" /><value value="gonal-f" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/95/001/025" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/95/001/026" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/95/001/027" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/95/001/021" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/95/001/031" /></identifier><identifier><system value="http://spor.ema.europa.eu/v2/marketing-authorisation-numbers" /><value value="EU/1/95/001/

2021-06-24 03:15:13,967 : List Bundle Creation Logger_3_T : List update successfully completed 9b70aea6-c211-43a1-8204-2c36aaea490d | H | CAP |  en | 3 | gonal-f_clean_ PACKAGE LEAFLET.json
2021-06-24 03:15:13,968 : Flow Logger HTML_1 : Completed list bundle update/addition | H | CAP |  en | HTML | gonal-f_clean.htm
2021-06-24 03:15:13,971 : Flow Logger HTML_1 : 3: Update/Add List Bundle,0.3051 Min,0.29467 MB,0.469493 MB,77.3%
 | H | CAP |  en | HTML | gonal-f_clean.htm
2021-06-24 03:15:13,974 : Flow Logger HTML_1 : Completed Processing Partitioned Jsons | H | CAP |  en | HTML | gonal-f_clean.htm
2021-06-24 03:15:13,977 : Flow Logger HTML_1 : 3: Completed,0.0 Min,0.149633 MB,0.151623 MB,77.3%
 | H | CAP |  en | HTML | gonal-f_clean.htm
2021-06-24 03:15:13,979 : Flow Logger HTML_1 : Final Metrics,11.8479 Min,0.0 MB,62.531705 MB,77.3%
 | H | CAP |  en | HTML | gonal-f_clean.htm


Metrics : 3: Update/Add List Bundle,0.3051 Min,0.29467 MB,0.469493 MB,77.3%

Metrics : 3: Completed,0.0 Min,0.149633 MB,0.151623 MB,77.3%

Metrics : Final Metrics,11.8479 Min,0.0 MB,62.531705 MB,77.3%



In [6]:
inputList = ['ELOCTA~H~CAP~bg~2021-06-09T08-18-48Z.zip',
 'ELOCTA~H~CAP~cs~2021-06-09T09-32-50Z.zip',
 'ELOCTA~H~CAP~da~2021-06-09T09-34-52Z.zip',
 'ELOCTA~H~CAP~de~2021-06-09T09-36-33Z.zip',
 'ELOCTA~H~CAP~el~2021-06-07T06-03-45Z.zip']

runAll(inputList)

2021-06-24 03:37:50,101 : Flow Logger HTML_o : Starting HTML Conversion To Json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-24 03:37:50,109 : Style Dictionary_A : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-24 03:37:50,190 : Style Dictionary_A : Qrd Section Keys Retrieved For Style Dictionary: КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА, ПРИЛОЖЕНИЕ II, A. ДАННИ ВЪРХУ ОПАКОВКАТА, Б.  ЛИСТОВКА | H | CAP |  bg | HTML | Elocta_clean.htm


ELOCTA~H~CAP~bg~2021-06-09T08-18-48Z.zip D:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\bg\2021-06-09T08-18-48Z D:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
Elocta_clean.htm
['D:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'ELOCTA', 'bg', '2021-06-09T08-18-48Z'] Elocta_clean.htm
2021-06-09T08-18-48Z bg ELOCTA CAP H
------------- D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.txt -----------------
D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\bg\2021-06-09T08-18-48Z\Elocta_clean.htm D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.json


2021-06-24 03:37:51,901 : Parser_L : Style Information Stored In File: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.txt | H | CAP |  bg | HTML | Elocta_clean.htm


!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!


2021-06-24 03:38:02,135 : Parser_L : Writing to file: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\bg\2021-06-09T08-18-48Z\outputJSON\Elocta_clean.json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-24 03:38:02,750 : Flow Logger HTML_o : Completed HTML Conversion To Json | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-24 03:38:02,752 : Flow Logger HTML_o : HTML Conversion To Json,0.2109 Min,7.776406 MB,19.400294 MB,79.4%
 | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-24 03:38:02,769 : Flow Logger HTML_o : Starting Json Split | H | CAP |  bg | HTML | Elocta_clean.htm
2021-06-24 03:38:02,775 : Style Dictionary_v : Reading style dictionary in file: rule_dictionary_bg.json | H | CAP |  bg | Json | Elocta_clean.json
2021-06-24 03:38:02,827 : Style Dictionary_v : Qrd Section Keys Retrieved For Style Dictionary: КРАТКА ХАРАКТЕРИСТИКА НА ПРОДУКТА, ПРИЛОЖЕНИЕ II, A. ДАННИ ВЪРХУ ОПАКОВКАТА, Б.  ЛИСТОВКА | H | CAP |  bg | Json | Elocta_clean.json
2021-06-24 03:3

...

Metrics : 3: Update/Add List Bundle,0.2608 Min,0.211574 MB,0.53975 MB,71.2%

Metrics : 3: Completed,0.0 Min,0.14956 MB,0.152213 MB,71.2%

Metrics : Final Metrics,5.3906 Min,0.0 MB,20.170449 MB,73.2%

ELOCTA~H~CAP~el~2021-06-07T06-03-45Z.zip D:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\el\2021-06-07T06-03-45Z D:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
Elocta_clean.htm
['D:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'ELOCTA', 'el', '2021-06-07T06-03-45Z'] Elocta_clean.htm
2021-06-07T06-03-45Z el ELOCTA CAP H


2021-06-24 04:01:08,792 : Style Dictionary_v : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ ΙΙ, Α. ΕΠΙΣΗΜΑΝΣΗ, Β. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | HTML | Elocta_clean.htm
2021-06-24 04:01:08,792 : Style Dictionary_v : Qrd Section Keys Retrieved For Style Dictionary: ΠΕΡΙΛΗΨΗ ΤΩΝ ΧΑΡΑΚΤΗΡΙΣΤΙΚΩΝ ΤΟΥ ΠΡΟΪΟΝΤΟΣ, ΠΑΡΑΡΤΗΜΑ ΙΙ, Α. ΕΠΙΣΗΜΑΝΣΗ, Β. ΦΥΛΛΟ ΟΔΗΓΙΩΝ ΧΡΗΣΗΣ | H | CAP |  el | HTML | Elocta_clean.htm


------------- D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.txt -----------------
D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\el\2021-06-07T06-03-45Z\Elocta_clean.htm D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.json


2021-06-24 04:01:11,123 : Parser_I : Style Information Stored In File: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\el\2021-06-07T06-03-45Z\outputJSON\Elocta_clean.txt | H | CAP |  el | HTML | Elocta_clean.htm


KeyError: 'ΠΑΡΑΡΤΗΜΑ ΙΙ'

In [ ]:
inputList = [ 'ELOCTA~H~CAP~en~2021-06-09T12-44-02Z.zip',
 'ELOCTA~H~CAP~es~2021-06-09T09-40-41Z.zip',
 'ELOCTA~H~CAP~et~2021-06-09T09-44-12Z.zip',
 'ELOCTA~H~CAP~fi~2021-06-09T09-51-23Z.zip',
 'ELOCTA~H~CAP~fr~2021-06-09T09-52-44Z.zip']

runAll(inputList)

2021-06-24 04:08:30,129 : Flow Logger HTML_n : Starting HTML Conversion To Json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-24 04:08:30,138 : Style Dictionary_N : Creating default style dictionary in file: rule_dictionary_en.json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-24 04:08:30,174 : Style Dictionary_N : Qrd Section Keys Retrieved For Style Dictionary: SUMMARY OF PRODUCT CHARACTERISTICS, ANNEX II, A. LABELLING , B. PACKAGE LEAFLET | H | CAP |  en | HTML | Elocta_clean.htm


ELOCTA~H~CAP~en~2021-06-09T12-44-02Z.zip D:\Projects\EMA\Repository\EMA EPI PoC\function_code\inputblob D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\en\2021-06-09T12-44-02Z D:\Projects\EMA\Repository\EMA EPI PoC\function_code\control
Already Present
Elocta_clean.htm
['D:', 'Projects', 'EMA', 'Repository', 'EMA EPI PoC', 'function_code', 'work', 'H', 'CAP', 'ELOCTA', 'en', '2021-06-09T12-44-02Z'] Elocta_clean.htm
2021-06-09T12-44-02Z en ELOCTA CAP H
------------- D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.txt -----------------
D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\en\2021-06-09T12-44-02Z\Elocta_clean.htm D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.json


2021-06-24 04:08:31,076 : Parser_u : Style Information Stored In File: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.txt | H | CAP |  en | HTML | Elocta_clean.htm


!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!
!!!!!!!!!!! IN TABLE !!!!!!!!!!!!!!!!!


2021-06-24 04:08:38,978 : Parser_u : Writing to file: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\en\2021-06-09T12-44-02Z\outputJSON\Elocta_clean.json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-24 04:08:39,506 : Flow Logger HTML_n : Completed HTML Conversion To Json | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-24 04:08:39,508 : Flow Logger HTML_n : HTML Conversion To Json,0.1563 Min,11.0488 MB,21.689423 MB,69.1%
 | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-24 04:08:39,523 : Flow Logger HTML_n : Starting Json Split | H | CAP |  en | HTML | Elocta_clean.htm
2021-06-24 04:08:39,530 : Style Dictionary_M : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Elocta_clean.json
2021-06-24 04:08:39,530 : Style Dictionary_M : Reading style dictionary in file: rule_dictionary_en.json | H | CAP |  en | Json | Elocta_clean.json
2021-06-24 04:08:39,572 : Style Dictionary_M : Qrd Section Keys Retrieved For Style Dictionary: S

...



Heading Not Found 
 ['nan,qSellele ravimile kohaldatakse täiendavat järelevalvet, mis võimaldab kiiresti tuvastada uut ohutusteavet. Tervishoiutöötajatel palutakse teatada kõigist võimalikest kõrvaltoimetest. Kõrvaltoimetest teatamise kohta vt lõik 4.8.>,2', '2.1,Üldkirjeldus,5', '2.2,Kvalitatiivne ja kvantitatiivne koostis,6', 'nan,Enne ravimi käsitsemist või manustamist tuleb järgida ettevaatusabinõusid,15', 'nan,Lapsed,21', 'nan,Rasedus,23', 'nan,Imetamine,24', 'nan,Fertiilsus,25', 'nan,Lapsed,31', 'nan,Farmakodünaamilised toimed,35', 'nan,Imendumine,39', 'nan,Jaotumine,40', 'nan,Biotransformatsioon,41', 'nan,Eritumine,42', 'nan,Lineaarsus/mittelineaarsus,43', 'nan,Farmakokineetilised/farmakodünaamilised toimed,44', 'nan,Keskkonnariski hindamine,46', 'nan,Kasutamine lastel,54', '11,DOSIMEETRIA,59', '12,RADIOFARMATSEUTILISE PREPARAADI VALMISTAMISE JUHEND,60']


dict_keys([])
Completed Heading Extraction For File
Metrics : 0: Heading Extraction,1.163 Min,3.573665 MB,7.6898 MB,70.1%


2021-06-24 04:20:32,270 : Flow Logger HTML_S : Completed Document Annotation | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-24 04:20:32,271 : Flow Logger HTML_S : 0: Document Annotation,0.0969 Min,0.049568 MB,0.24744 MB,70.3%
 | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-24 04:20:32,274 : Flow Logger HTML_S : Starting Extracting Content Between Heading | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-24 04:20:32,284 : ExtractContentBetween_0_b : Cleaning Match Results | H | CAP |  et | 0 | Elocta_clean_SmPC.json
2021-06-24 04:20:32,294 : ExtractContentBetween_0_b : Finished Cleaning Match Results | H | CAP |  et | 0 | Elocta_clean_SmPC.json
2021-06-24 04:20:32,426 : Flow Logger HTML_S : Completed Extracting Content Between Heading | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-24 04:20:32,427 : Flow Logger HTML_S : 0: Content Extraction,0.0026 Min,1.155869 MB,7.711439 MB,70.3%
 | H | CAP |  et | HTML | Elocta_clean.htm
2021-06-24 04:20:32,480 : XmlGeneration_0_Z : PMS/OMS A

Missing Key 'entry' in the EU/1/15/1046/008 regulated authorization API output
None
Completed Document Annotation
Metrics : 0: Document Annotation,0.0969 Min,0.049568 MB,0.24744 MB,70.3%

Starting Extracting Content Between Heading For File :- Elocta_clean_SmPC.json
File being processed: D:\Projects\EMA\Repository\EMA EPI PoC\function_code\work\H\CAP\ELOCTA\et\2021-06-09T09-44-12Z\partitionedJSONs\Elocta_clean_SmPC.json
--------------------------------------------
Completed Extracting Content Between Heading
Metrics : 0: Content Extraction,0.0026 Min,1.155869 MB,7.711439 MB,70.3%



2021-06-24 04:20:32,482 : XmlGeneration_0_Z : Initiating XML Generation | H | CAP |  et | 0 | Elocta_clean_SmPC.json
2021-06-24 04:20:32,861 : XmlGeneration_0_Z : Writing to File:Elocta_clean_SmPC.xml | H | CAP |  et | 0 | Elocta_clean_SmPC.json
2021-06-24 04:20:32,868 : Flow Logger HTML_S : 0: Generate XML,0.0073 Min,2.738825 MB,6.075307 MB,70.3%
 | H | CAP |  et | HTML | Elocta_clean.htm


Metrics : 0: Generate XML,0.0073 Min,2.738825 MB,6.075307 MB,70.3%



In [ ]:
inputList = ['ELOCTA~H~CAP~hr~2021-06-09T09-54-07Z.zip',
 'ELOCTA~H~CAP~hu~2021-06-09T09-55-10Z.zip',
 'ELOCTA~H~CAP~is~2021-06-09T09-57-48Z.zip',
 'ELOCTA~H~CAP~it~2021-06-09T09-59-11Z.zip',
 'ELOCTA~H~CAP~lt~2021-06-09T10-00-51Z.zip']

runAll(inputList)

In [ ]:
inputList = ['ELOCTA~H~CAP~lv~2021-06-09T10-02-03Z.zip',
 'ELOCTA~H~CAP~mt~2021-06-09T10-03-45Z.zip',
 'ELOCTA~H~CAP~nl~2021-06-09T10-04-39Z.zip',
 'ELOCTA~H~CAP~no~2021-06-09T10-07-44Z.zip',
 'ELOCTA~H~CAP~pl~2021-06-09T10-08-48Z.zip']

runAll(inputList)

In [ ]:
inputList = [ 'ELOCTA~H~CAP~pt~2021-06-09T10-09-45Z.zip',
 'ELOCTA~H~CAP~ro~2021-06-09T10-12-05Z.zip',
 'ELOCTA~H~CAP~sk~2021-06-09T10-12-51Z.zip',
 'ELOCTA~H~CAP~sl~2021-06-09T10-13-47Z.zip',
 'ELOCTA~H~CAP~sv~2021-06-09T10-14-39Z.zip']

runAll(inputList)

In [ ]:

inputList = ['ELOCTA~H~CAP~el~2021-06-07T06-03-45Z.zip']

runAll(inputList)

In [ ]:
inputList = ['ELOCTA~H~CAP~bg~2021-06-09T08-18-48Z.zip',
 'ELOCTA~H~CAP~cs~2021-06-09T09-32-50Z.zip',
 'ELOCTA~H~CAP~da~2021-06-09T09-34-52Z.zip',
 'ELOCTA~H~CAP~de~2021-06-09T09-36-33Z.zip',
 'ELOCTA~H~CAP~el~2021-06-07T06-03-45Z.zip',
 'ELOCTA~H~CAP~en~2021-06-09T12-44-02Z.zip',
 'ELOCTA~H~CAP~es~2021-06-09T09-40-41Z.zip',
 'ELOCTA~H~CAP~et~2021-06-09T09-44-12Z.zip',
 'ELOCTA~H~CAP~fi~2021-06-09T09-51-23Z.zip',
 'ELOCTA~H~CAP~fr~2021-06-09T09-52-44Z.zip',
 'ELOCTA~H~CAP~hr~2021-06-09T09-54-07Z.zip',
 'ELOCTA~H~CAP~hu~2021-06-09T09-55-10Z.zip',
 'ELOCTA~H~CAP~is~2021-06-09T09-57-48Z.zip',
 'ELOCTA~H~CAP~it~2021-06-09T09-59-11Z.zip',
 'ELOCTA~H~CAP~lt~2021-06-09T10-00-51Z.zip',
 'ELOCTA~H~CAP~lv~2021-06-09T10-02-03Z.zip',
 'ELOCTA~H~CAP~mt~2021-06-09T10-03-45Z.zip',
 'ELOCTA~H~CAP~nl~2021-06-09T10-04-39Z.zip',
 'ELOCTA~H~CAP~no~2021-06-09T10-07-44Z.zip',
 'ELOCTA~H~CAP~pl~2021-06-09T10-08-48Z.zip',
 'ELOCTA~H~CAP~pt~2021-06-09T10-09-45Z.zip',
 'ELOCTA~H~CAP~ro~2021-06-09T10-12-05Z.zip',
 'ELOCTA~H~CAP~sk~2021-06-09T10-12-51Z.zip',
 'ELOCTA~H~CAP~sl~2021-06-09T10-13-47Z.zip',
 'ELOCTA~H~CAP~sv~2021-06-09T10-14-39Z.zip']

runAll(inputList)